# Cherry Pick script with PyLabRobot for Hamilton

# questions 

Does lld work? 

# To do

Copy Hamilton 223 specs from lab to fit in PLR
Write code for cherry pick
Simulate code on PLR simulator
Test code with water. 
Test with foil seal. Can i get it to override LLD?
Implement in cherry_pick git repo 

Test if i can split into the three parts. 

In [1]:
# start inputs
t_list = [['10', 'A1', '9', 'A1', '100', 'FSH0345O'], ['10', 'B1', '9', 'B1', '100', 'FSH0345O'], ['10', 'C1', '9', 'C1', '100', 'FSH0345O'], ['10', 'D1', '9', 'D1', '100', 'FSH0345O'], ['10', 'E1', '9', 'E1', '100', 'FSH0345O'], ['10', 'F1', '9', 'F1', '100', 'FSH0345O'], ['10', 'G1', '9', 'G1', '100', 'FSH0345O'], ['10', 'H1', '9', 'H1', '100', 'FSH0345O'], ['10', 'A2', '9', 'A2', '100', 'FSH0345O'], ['10', 'B2', '9', 'B2', '100', 'FSH0345O'], ['10', 'C2', '9', 'C2', '100', 'FSH0345O'], ['10', 'D2', '9', 'D2', '100', 'FSH0345O'], ['10', 'E2', '9', 'E2', '100', 'FSH0345O'], ['10', 'F2', '9', 'F2', '100', 'FSH0345O'], ['10', 'G2', '9', 'G2', '100', 'FSH0345O'], ['10', 'H2', '9', 'H2', '100', 'FSH0345O'], ['10', 'A3', '9', 'A3', '100', 'FSH0345O'], ['10', 'B3', '9', 'B3', '100', 'FSH0345O'], ['10', 'C3', '9', 'C3', '100', 'FSH0345O'], ['10', 'D3', '9', 'D3', '100', 'FSH0345O'], ['10', 'E3', '9', 'E3', '100', 'FSH0345O'], ['10', 'F3', '9', 'F3', '100', 'FSH0345O'], ['10', 'G3', '9', 'G3', '100', 'FSH0345O'], ['10', 'H3', '9', 'H3', '100', 'FSH0345O'], ['10', 'A4', '9', 'A4', '100', 'FSH0345O'], ['10', 'B4', '9', 'B4', '100', 'FSH0345O'], ['10', 'C4', '9', 'C4', '100', 'FSH0345O'], ['10', 'D4', '9', 'D4', '100', 'FSH0345O'], ['10', 'E4', '9', 'E4', '100', 'FSH0345O'], ['10', 'F4', '9', 'F4', '100', 'FSH0345O'], ['10', 'G4', '9', 'G4', '100', 'FSH0345O'], ['10', 'H4', '9', 'H4', '100', 'FSH0345O'], ['10', 'A5', '9', 'A5', '100', 'FSH0345O'], ['10', 'B5', '9', 'B5', '100', 'FSH0345O'], ['10', 'C5', '9', 'C5', '100', 'FSH0345O'], ['10', 'D5', '9', 'D5', '100', 'FSH0345O'], ['10', 'E5', '9', 'E5', '100', 'FSH0345O'], ['10', 'F5', '9', 'F5', '100', 'FSH0345O'], ['7', 'A1', '9', 'G5', '100', 'FSH034AP'], ['7', 'B1', '9', 'H5', '100', 'FSH034AP'], ['7', 'C1', '9', 'A6', '100', 'FSH034AP'], ['7', 'E1', '9', 'B6', '100', 'FSH034AP'], ['7', 'A2', '9', 'C6', '100', 'FSH034AP'], ['7', 'D2', '9', 'D6', '100', 'FSH034AP'], ['7', 'E2', '9', 'E6', '100', 'FSH034AP'], ['7', 'F2', '9', 'F6', '100', 'FSH034AP'], ['7', 'G2', '9', 'G6', '100', 'FSH034AP'], ['7', 'B3', '9', 'H6', '100', 'FSH034AP'], ['7', 'E3', '9', 'A7', '100', 'FSH034AP'], ['7', 'F3', '9', 'B7', '100', 'FSH034AP'], ['7', 'G3', '9', 'C7', '100', 'FSH034AP'], ['7', 'A4', '9', 'D7', '100', 'FSH034AP'], ['7', 'C4', '9', 'E7', '100', 'FSH034AP'], ['7', 'F4', '9', 'F7', '100', 'FSH034AP'], ['7', 'H4', '9', 'G7', '100', 'FSH034AP'], ['7', 'A5', '9', 'H7', '100', 'FSH034AP'], ['7', 'B5', '9', 'A8', '100', 'FSH034AP'], ['7', 'C5', '9', 'B8', '100', 'FSH034AP'], ['7', 'D5', '9', 'C8', '100', 'FSH034AP'], ['7', 'E5', '9', 'D8', '100', 'FSH034AP'], ['7', 'F5', '9', 'E8', '100', 'FSH034AP'], ['7', 'G5', '9', 'F8', '100', 'FSH034AP'], ['7', 'H5', '9', 'G8', '100', 'FSH034AP'], ['7', 'E6', '9', 'H8', '100', 'FSH034AP'], ['7', 'F6', '9', 'A9', '100', 'FSH034AP'], ['7', 'G6', '9', 'B9', '100', 'FSH034AP'], ['7', 'H6', '9', 'C9', '100', 'FSH034AP'], ['7', 'A7', '9', 'D9', '100', 'FSH034AP'], ['7', 'B7', '9', 'E9', '100', 'FSH034AP'], ['7', 'C7', '9', 'F9', '100', 'FSH034AP'], ['7', 'D7', '9', 'G9', '100', 'FSH034AP'], ['7', 'E7', '9', 'H9', '100', 'FSH034AP'], ['7', 'F7', '9', 'A10', '100', 'FSH034AP'], ['7', 'G7', '9', 'B10', '100', 'FSH034AP'], ['7', 'H7', '9', 'C10', '100', 'FSH034AP'], ['7', 'A8', '9', 'D10', '100', 'FSH034AP'], ['7', 'B8', '9', 'E10', '100', 'FSH034AP'], ['7', 'C8', '9', 'F10', '100', 'FSH034AP'], ['7', 'D8', '9', 'G10', '100', 'FSH034AP'], ['7', 'E8', '9', 'H10', '100', 'FSH034AP'], ['7', 'F8', '9', 'A11', '100', 'FSH034AP'], ['7', 'G8', '9', 'B11', '100', 'FSH034AP'], ['7', 'H8', '9', 'C11', '100', 'FSH034AP'], ['7', 'A9', '9', 'D11', '100', 'FSH034AP'], ['7', 'B9', '9', 'E11', '100', 'FSH034AP'], ['7', 'C9', '9', 'F11', '100', 'FSH034AP'], ['7', 'F9', '9', 'G11', '100', 'FSH034AP'], ['7', 'G9', '9', 'H11', '100', 'FSH034AP'], ['7', 'A10', '9', 'A12', '100', 'FSH034AP'], ['7', 'B10', '9', 'B12', '100', 'FSH034AP'], ['7', 'C10', '9', 'C12', '100', 'FSH034AP'], ['7', 'D10', '9', 'D12', '100', 'FSH034AP']]
control_input = {'source_plate': 'biorad_96_wellplate_200ul_pcr', 'dest_plate': 'usascientific_96_wellplate_2.4ml_deep', 'right_pipette': 'p300_single', 'left_pipette': 'p10_single'}

# Settup Simulator or Backend

In [2]:
# simulate = False
simulate = True


from pylabrobot.liquid_handling import LiquidHandler

if simulate:
  # from pylabrobot.liquid_handling.backends.simulation.simulator_backend import SimulatorBackend
  from pylabrobot.liquid_handling import LiquidHandler
  from pylabrobot.liquid_handling.backends import ChatterBoxBackend
  from pylabrobot.visualizer.visualizer import Visualizer
else:
  from pylabrobot.liquid_handling.backends import STAR


# from pylabrobot.resources.hamilton import HamiltonSTARDeck
from pylabrobot.resources.hamilton import STARDeck
# from pylabrobot.resources.hamilton import STARLetDeck

STAR_SIZE_X=1900
STAR_SIZE_Y=653.5
STAR_SIZE_Z=900
if simulate:
  # sb = SimulatorBackend(open_browser=False)
  # lh = LiquidHandler(backend=sb, deck=HamiltonSTARDeck(num_rails=55, size_x=STAR_SIZE_X, size_y=STAR_SIZE_Y, size_z=STAR_SIZE_Z))
  lh = LiquidHandler(backend=ChatterBoxBackend(), deck=STARDeck())
  vis = Visualizer(resource=lh)
else:
  backend = STAR()
  lh = LiquidHandler(backend=backend, deck=STARDeck())


await lh.setup()

if simulate:
  await vis.setup()


# if simulate:
#   sb.wait_for_connection()


# Work table sesources
from pylabrobot.resources import (
  TIP_CAR_480_A00,
    # PLT_CAR_L5AC_A00,
    # Cos_96_DW_1mL,
    # HTF_L
  TIP_CAR_NTR_A00,
  # MFXCarrier, # Think is: MFX_CAR_3TipBC_2NTR1
  # ShakerCarrier, # think is: CVS_Shaker0_A00
  PLT_CAR_L5AC,
)


# labware resources:
from pylabrobot.resources import (
  TIP_50ul_L,
  corning_96_wellplate_360ul_flat, # OT plate type
  Cos_96_FL # corning alternative to OT plate
)


from pylabrobot.resources.carrier import MFXCarrier, create_homogeneous_carrier_sites, Coordinate

## there are no tube runners in PLR yet.


from pylabrobot.resources.trash import Trash

trash = lh.deck.get_trash_area()
# n must match with the number of pipets
n=1

offsets = trash.get_2d_center_offsets(n=n)


Setting up the robot.
Resource deck was assigned to the robot.
Resource trash was assigned to the robot.
Websocket server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


# Settup Deck

In [3]:
# PLT_CAR_L5AC

## Tip positioning
tip_car = TIP_CAR_480_A00(name='tip_carrier') ### OPS OPS USING THE BIG TIP CARRIER!!!!
tip_car[1] = TIP_50ul_L(name='tip01')
tip_car[2] = TIP_50ul_L(name='tip02')
lh.deck.assign_child_resource(tip_car, rails=7)



plt_car = PLT_CAR_L5AC(name='PLT_CAR_L5AC_1')
# plt_car[3] = corning_96_wellplate_360ul_flat(name='plt01')
# plt_car[4] = corning_96_wellplate_360ul_flat(name='plt02')
plt_car[3] = Cos_96_FL(name='plt01')
plt_car[4] = Cos_96_FL(name='plt02')

lh.deck.assign_child_resource(plt_car, rails=27)

plt_car = PLT_CAR_L5AC(name='PLT_CAR_L5AC_2')
lh.deck.assign_child_resource(plt_car, rails=33)

plt_car_cooler = PLT_CAR_L5AC(name='PLT_CAR_L5AC_cool_1')
lh.deck.assign_child_resource(plt_car_cooler, rails=43)

plt_car_cooler = PLT_CAR_L5AC(name='PLT_CAR_L5AC_cool_2')
lh.deck.assign_child_resource(plt_car_cooler, rails=49)

Resource tip_carrier was assigned to the robot.
Resource PLT_CAR_L5AC_1 was assigned to the robot.
Resource PLT_CAR_L5AC_2 was assigned to the robot.
Resource PLT_CAR_L5AC_cool_1 was assigned to the robot.
Resource PLT_CAR_L5AC_cool_2 was assigned to the robot.


In [4]:
# MFXCarrier

# mfx_carrier = MFXCarrier(name="Example MFX Carrier", size_x=10.0, size_y=5.0, size_z=3.0)

# # You can access attributes of the instance, for example:
# print(mfx_carrier.name)  # Output: Example MFX Carrier

# # plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
# mfx_carrier[0] = Cos_96_DW_1mL(name='test_01')
# mfx_carrier[1] = Cos_96_DW_1mL(name='test_02')
# # lh.deck.assign_child_resource(plt_car, rails=15)



## OK SO THIS IS CORRECT BUT THE DIMENSIONS ARE CLEARLY WRONG
# from pylabrobot.resources.carrier import MFXCarrier, create_homogeneous_carrier_sites, Coordinate
# Define the carrier name and dimensions
name = "my_mfx_carrier"
size_x = 100.0
size_y = 100.0
size_z = 20.0

# Define the carrier sites
site_locations = [Coordinate(x=0, y=0), Coordinate(x=0, y=10), Coordinate(x=10, y=0), Coordinate(x=10, y=10)]
site_size_x = 10.0
site_size_y = 10.0
sites = create_homogeneous_carrier_sites(site_locations, site_size_x, site_size_y)

# Create an instance of the MFXCarrier class
mfx_carrier = MFXCarrier(name, size_x, size_y, size_z, sites)

lh.deck.assign_child_resource(mfx_carrier, rails=13)

Resource my_mfx_carrier was assigned to the robot.


In [6]:
# dummy CSV_Shaker0

## OK SO THIS IS CORRECT BUT THE DIMENSIONS ARE CLEARLY WRONG
from pylabrobot.resources.carrier import ShakerCarrier, create_homogeneous_carrier_sites, Coordinate
# Define the carrier name and dimensions
name = "my_shaker_carrier"
# size_x = 100.0 #157.5
# size_y = 100.0 #497.0
# size_z = 20 #90.0

# Define the carrier sites
site_locations = [Coordinate(x=0, y=0), Coordinate(x=0, y=10)]
site_size_x = 10 #154
site_size_y = 10 #95

size_x=157.5,
size_y=497.0,
size_z=90.0,
site_locations=create_homogeneous_carrier_sites([
    Coordinate(154.0, 95.0, 114.95),
    Coordinate(154.0, 86.0, 114.95),
    Coordinate(154.0, 95.0, 114.95),
    Coordinate(154.0, 298.0, 114.95),
    Coordinate(154.0, 394.0, 114.95)
  ],
  site_size_x=122.4,
  site_size_y=82.6,
),


sites = create_homogeneous_carrier_sites(site_locations, site_size_x, site_size_y)

# Create an instance of the MFXCarrier class
shaker_carrier = ShakerCarrier(name, size_x, size_y, size_z, sites)

lh.deck.assign_child_resource(shaker_carrier, rails=1, reassign=True)


Resource my_shaker_carrier was assigned to the robot.


# Simulation create liquids in plates and fill tips

In [6]:
# plt_car = PLT_CAR_L5AC(name='PLT_CAR_L5AC_1')
# plt_car[3] = corning_96_wellplate_360ul_flat(name='plt01')
# plt_car[4] = corning_96_wellplate_360ul_flat(name='plt02')

tiprack = lh.get_resource("tip01")
dist1 = lh.get_resource('plt02')
source_plate = lh.get_resource('plt01')

if simulate:
  # define Tips
  tiprack.fill()
  plate_1_liquids = [[(None, 50)]]*96
  # vis.adjust_wells_liquids(source_plate, liquids=plate_1_liquids)
  source_plate.set_well_liquids(plate_1_liquids)

# Liquid Commands

In [5]:
# ## Generate all well positions to pick tips from
# rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
# columns = range(1, 13)
# tip_slot = [f"{row}{col}" for row in rows for col in columns]
# print(tip_slot)

# well_names_sorted = sorted(tip_slot, key=lambda x: (x[1:], x[0]))
# print(well_names_sorted)

In [6]:
tip_slot = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9', 'A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10', 'A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11', 'A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12']

In [7]:
# source_slot = t_list[count][0]
# source_well = t_list[count][1]
# dist_slot = t_list[count][2]
# dist_well = t_list[count][3]
# volume = float(t_list[0][4])
# source_plate = find_plate_name(input_slots, int(source_slot))
# print('source_plate', source_plate)
# print('source_well', source_well)
# print(t_list[count])


# await lh.pick_up_tips(tiprack['C1'])
await lh.pick_up_tips(tiprack["C1:E1"])


# await lh.aspirate(source_plate['H1'], vols=[50.0])
# await lh.aspirate(source1[source_well], vols=volume)

source_plate = lh.get_resource('plt01')
await lh.aspirate(source_plate["A1:C1"], vols=[20.0, 40.0, 10.0], swap_speed=100, homogenization_speed=1000)

await lh.dispense(dist1["A1:C1"], vols=[20.0, 50.0, 10.0], swap_speed=100, homogenization_speed=1000, mix_speed=1000)

# await lh.drop_tips(tip_spots=[trash]*1, offsets=offsets)
await lh.drop_tips(tiprack["C1"])


await lh.drop_tips(tiprack["E1"], use_channels=[2])


10


Plate(name=source_1, size_x=127.76, size_y=85.47, size_z=14.22, location=(000.000, 271.500, 000.000))

In [10]:
print(lh.deck.summary())


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: source_1    | 11: tips_01     | 12: trash_co... |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: source_2    |  8: tips_02     |  9: dist1       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: source_3    |  5: Empty       |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: source_4    |  2: Empty       |  3: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



source_plate Plate(name=source_1, size_x=127.76, size_y=85.47, size_z=14.22, location=(000.000, 271.500, 000.000))
source_well A1
['10', 'A1', '9', 'A1', '100', 'FSH0345O']


source_plate Plate(name=source_1, size_x=127.76, size_y=85.47, size_z=14.22, location=(000.000, 271.500, 000.000))
source_well B1
['10', 'B1', '9', 'B1', '100', 'FSH0345O']


['10', 'A1', '9', 'A1', '100', 'FSH0345O']
92


In [ ]:
print(lh.deck.summary())


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: plate_02    | 11: plate_01    | 12: trash_co... |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: tips_02     |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: Empty       |  3: tips_01     |
|                 |                 |                 |
+-----------------+-----------------+-----------------+



# Shutdown

In [ ]:
await lh.stop()

# Alternative tip picking test


# Building Hamilton deck for Hamilton in B221 R032